In [9]:
#importing libraries
import matplotlib.pyplot as plt
import json
from flask import Flask, render_template, redirect, url_for, request, session, g
from flask_login import login_required, LoginManager, UserMixin, login_user, current_user
from functools import wraps
import sqlite3
import os


app = Flask(__name__)
app.secret_key = 'secret_key'

def get_db_connection():
    conn = sqlite3.connect('expat_project.db')
    conn.row_factory = sqlite3.Row
    conn.set_trace_callback(print)
    return conn

@app.before_request
def before_request():
    g.user = None
    if 'user_id' in session:
        conn = get_db_connection()
        g.user = conn.execute("SELECT * FROM users WHERE id = ?", (session['user_id'],)).fetchone()
        conn.close()

@app.route("/")
def Landing_Page():
    try:
        return render_template("home.html")
    except Exception as e:
        return str(e)

@app.route("/login", methods=['POST','GET'])
def login():
    if request.method == 'POST':
     
        connection = get_db_connection()
        cursor = connection.cursor()

        email = request.form['email']
        password = request.form['psw']

        cursor.execute('SELECT users.id, consultants.id as "consultant_id", users.email, users.pass_hash FROM users LEFT JOIN consultants ON consultants.user_id = users.id where users.email=? AND users.pass_hash=?', (email, password))

        results = cursor.fetchone()

        if results:
            session['user_id'] = results['id']
            user = results[0]
            if results["consultant_id"] is not None:
                return redirect("/consultants/" + str(results['consultant_id']) + "/cases")
            
            else:
                return render_template("Logged.html", user = user)
            

        else:
            return render_template("LoginError.html")


@app.route("/logout")
def logout():
    session.pop('logged_in', None)
    return render_template("logout.html", message = "You have been successfully logged out.")




@app.route("/registered", methods=['POST','GET'])
def registered():
    if request.method == 'POST':
        try:
            first_name = request.form['name']
            last_name = request.form['surname']
            email = request.form['email']
            pass_hash = request.form['psw']
            
            with sqlite3.connect('expat_project.db') as con:
                c = con.cursor()
                c.execute("SELECT * FROM users WHERE email =?",(email,))
                existing_user = c.fetchone()
                if existing_user:
                    msg = "Email already exists, please use another email"
                    return render_template("Registered.html", msg=msg)
                else:
                    c.execute("INSERT INTO users (first_name,last_name,email,pass_hash) VALUES (?,?,?,?)",(first_name,last_name,email,pass_hash))
                    c.execute("SELECT id FROM users WHERE email=?",(email,))
                    user_id = c.fetchone()[0]
                    con.commit()
                    msg = "Welcome to Expat Assistance "+first_name
                    return redirect("/users/{}/select_application".format(user_id))
        except Exception as e:
            con.rollback()
            msg = "Got back error to register " + str(e)
        finally:
            con.close()
          
            
              
########################################################################################


#page to select application type


@app.route("/users/<int:user_id>/select_application")
def select_application(user_id):
    if g.user and g.user['id'] == user_id:
        conn = get_db_connection()
        rows = conn.execute("SELECT id, title FROM application_types").fetchall()
        conn.close()
        return render_template('select_application_type.html', rows = rows, user_id = user_id) 
    
    else:
        return redirect('/')

#page and query to get existing cases for a user 

@app.route("/users/<int:user_id>/cases")
def user_cases(user_id):
    if g.user and g.user['id'] == session['user_id'] and g.user['id'] == user_id:
        conn = get_db_connection()
        rows = conn.execute("""SELECT
            t.title,
            u.first_name as "user_first_name",
            u.last_name as "user_last_name",
            a.created_at,
            a.updated_at,
            c.first_name as "consultant_first_name",
            c.last_name as "consultant_last_name",
            c.company_name,
            a.status 
        FROM applications AS a
        LEFT JOIN application_types as t 
        ON a.application_type_id = t.id
        LEFT JOIN users AS u
        ON a.creator_id = u.id
        LEFT JOIN consultants AS c
        ON a.assigned_consultant_id = c.id
        WHERE a.creator_id = ?""",(user_id,)).fetchall()
        conn.close()
        return render_template('user_cases.html', rows = rows)
    
    else:
        return redirect('/')




#page to start a new application

@app.route("/users/<int:user_id>/applications/<application_type_id>/create")
def start_application(user_id, application_type_id):
    if g.user and g.user['id'] == session['user_id'] and g.user['id'] == user_id:
        conn = get_db_connection()
        rows = conn.execute('SELECT title, id, required FROM questions WHERE application_type_id = ' + application_type_id ).fetchall()
        conn.close()
        return render_template('start_application.html', questions = rows, user_id = user_id, application_type_id = application_type_id)  
    
    else:
        return redirect('/')



@app.route("/users/<int:user_id>/applications/<application_type_id>/submit",  methods=['POST'])
def submit_application(user_id, application_type_id):
    if g.user and g.user['id'] == user_id:
        response = request.form
        application_id = None
        msg = ''
        try:
            con = sqlite3.connect('expat_project.db')
            c = con.cursor()
            application_id = c.execute("INSERT INTO applications (application_type_id, creator_id) VALUES (?,?)",(application_type_id,user_id)).lastrowid
            print("inserted application with id: {}".format(application_id))
            for question_id in response:
                application_response_id = c.execute("INSERT INTO application_responses  (question_id, application_id, response) VALUES (?, ?, ?)", (question_id, application_id, response[question_id])).lastrowid
                print("inserted application response with id: {}".format(application_response_id))
            con.commit()
            msg = "Successfully submitted application with responses"

        except Exception as e:
            print("rollbacking the transaction {}!!".format(e))
            msg = "Incorrect submission. {}".format(e)
            con.rollback()

        finally:
            con.close()
            return render_template('submit_application.html', response = response, application_id = application_id, msg = msg, user_id = user_id)
    else:
        return redirect('/')
        

#page for consulatnts to 

@app.route("/consultants/<int:consultant_id>/cases")
def consultant_cases(consultant_id):
    if g.user and g.user['id'] == consultant_id:
        conn = get_db_connection()
        rows = conn.execute("""SELECT
         a.id, 
         t.title,
         u.first_name AS "user_first_name",
         u.last_name AS "user_last_name",
         a.created_at,
         a.updated_at, 
         c.first_name AS "consultant_first_name", 
         c.last_name AS "consultant_last_name",
         c.company_name, 
         a.status
    FROM applications as a
    LEFT JOIN application_types as t
    ON a.application_type_id = t.id
    LEFT JOIN users as u
    ON 	a.creator_id = u.id
    LEFT JOIN consultants as c
    ON 	a.assigned_consultant_id = c.id 
    WHERE a.assigned_consultant_id = ?""", (consultant_id,)).fetchall()
        conn.close()

        return render_template('consultant_cases.html', rows = rows, consultant_id = consultant_id) 
    
    else:
        return redirect('/')



@app.route("/consultants/<int:consultant_id>/cases/<application_id>", methods=["GET", "POST"])
def consultant_case(consultant_id, application_id):
    if g.user and g.user['id'] == consultant_id:
        msg = ''
        if request.method == 'POST':
            try:
                application_status = request.form['application_status']

                with sqlite3.connect('expat_project.db') as con:
                    c=con.cursor()
                    c.execute("UPDATE applications SET status = ? WHERE id = ?",(application_status,application_id))

                    con.commit()
                    msg = "Updated application status! "

    #When the page is refreshed, submit button does not work
            except Exception as e:
                con.rollback()
                msg = "Got back error updating status: " + str(e)

            finally:
                con.close()


        conn = get_db_connection()
        ## Adding consultant_id as WHERE clause to make sure we only get application if consultant is associated with it
        rows = conn.execute("""SELECT
         a.id, 
         t.title,
         u.first_name AS "user_first_name",
         u.last_name AS "user_last_name",
         a.created_at,
         a.updated_at, 
         c.first_name AS "consultant_first_name", 
         c.last_name AS "consultant_last_name",
         c.company_name, 
         a.status
    FROM applications as a
    LEFT JOIN application_types as t
    ON a.application_type_id = t.id
    LEFT JOIN users as u
    ON 	a.creator_id = u.id
    LEFT JOIN consultants as c
    ON 	a.assigned_consultant_id = c.id 
    WHERE a.assigned_consultant_id = ? AND a.id = ? """,(consultant_id, application_id)).fetchall()

        responses = conn.execute("""SELECT
            ar.*,
            q.title
        FROM application_responses as ar
        LEFT JOIN questions q ON q.id = ar.question_id
        WHERE ar.application_id = ? """, (application_id)).fetchall()
        conn.close()
        return render_template('consultant_case.html', application = rows[0], msg = msg, responses = responses, consultant_id = consultant_id) 
    
    else:
        return redirect('/')

@app.route("/About")
def index():
    connection = sqlite3.connect('expat_project.db')
    cursor = connection.cursor()
    cursor.execute("SELECT status, COUNT(*) FROM applications GROUP BY status")
    result = cursor.fetchall()
    connection.close()
    labels = []
    values = []
    for item in result:
        labels.append(item[0])
        values.append(item[1])
    total_count = sum(values)
    plt.bar(labels, values)
    plt.xlabel('Status')
    plt.ylabel('Count')
    plt.title('Finished, Submitted, In Progress')
    plt.savefig('static/status_graph.png')
    plt.close()
    return render_template('About.html')



app.run(host='localhost', port=8081, debug=True, use_reloader=False) #running app which is Flask at localhost at port 8081





 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://localhost:8081/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2023 12:27:32] "GET / HTTP/1.1" 200 -


SELECT * FROM users WHERE id = 5


127.0.0.1 - - [22/Jan/2023 12:27:37] "POST /login HTTP/1.1" 200 -


SELECT * FROM users WHERE id = 5
SELECT users.id, consultants.id as "consultant_id", users.email, users.pass_hash FROM users LEFT JOIN consultants ON consultants.user_id = users.id where users.email='oliverpowers@gnmail.com' AND users.pass_hash='12365'


...

127.0.0.1 - - [22/Jan/2023 12:29:03] "POST /login HTTP/1.1" 500 -


SELECT * FROM users WHERE id = 5
SELECT users.id, consultants.id as "consultant_id", users.email, users.pass_hash FROM users LEFT JOIN consultants ON consultants.user_id = users.id where users.email='oliverpowers@gnmail.com' AND users.pass_hash='12365'


Traceback (most recent call last):
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2450, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1867, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mraer\anaconda3\Lib\site-pa

SELECT * FROM users WHERE id = 5
SELECT users.id, consultants.id as "consultant_id", users.email, users.pass_hash FROM users LEFT JOIN consultants ON consultants.user_id = users.id where users.email='oliverpowers@gnmail.com' AND users.pass_hash='12365'


Traceback (most recent call last):
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2450, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1867, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mraer\anaconda3\Lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mraer\anaconda3\Lib\site-pa